In [23]:
# Import bigwig data 

# Function for importing bigWig data (coverage and signal coverage data)
FeatureBigWig<-function(x) {
	coverage<-c()
	for(i in 1:length(x)) {
		 coverage<- c(coverage, list(data.frame(import(x[[i]]))))		
		}
	coverage
}

# Merging the coverage and signal coverage data of the same sample together

combine<-function(x) {
    k<-c()
    for(i in seq(from=1, to=length(x), by=2)) {
        k<- c(k, list(data.frame(merge(x[[i]], x[[i+1]], 
                by.x=c('seqnames', 'start', 'end'), by.y=c('seqnames', 'start', 'end')))))
        }
    k
}

# Discarding the bases that have coverage below 10X and have more than 99.9th percentile of coverage in each sample

filter<-function(cg) {
    flt<-c()
        for(i in 1:length(cg)) { 
        flt<-c(flt, list(data.frame(cg[[i]][cg[[i]]$score.y <= quantile(cg[[i]]$score.y, .999)&cg[[i]]$score.y>=10,])))
    }
    flt
}

# Merging all the samples together, path is the the pathway where your data is stored 
# ("/home/.../xxxx"))

ImportBlueprint<-function(path) {
    library(rtracklayer)

    files <- dir(path,  full.names = TRUE)

    Feature.cell<-FeatureBigWig(files)

    merge.cell<-combine(Feature.cell)

    filter.merge.cell<-filter(merge.cell)
    
    names.files <- c(names(filter.merge.cell[[1]])[1:3],unique(gsub('.CPG.*', '', dir(path))))

    dat =filter.merge.cell[[1]]
    
    print(length(filter.merge.cell[[1]]$seqnames))

for(i in 2:length(filter.merge.cell)) {
    dat <- merge(dat, filter.merge.cell[[i]], by.x=c('seqnames', 'start', 'end'), by.y=c('seqnames', 'start', 'end'))
        print(length(filter.merge.cell[[i]]$seqnames))
            }
    a<-dat[,1:3]
    for(j in seq(from=6, to=ncol(dat), by=6)) {
        a<-cbind(a,dat[, j])
        }
    colnames(a)<-names.files 
    a
}

    
    
                     
                     